<a href="https://colab.research.google.com/github/AdnaneMOUZAOUI/Analyse-de-sentiments-machine-learning/blob/Reviews-Analysis/Word_embedding_2_modele_entrain%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install pandas matplotlib seaborn nltk scikit-learn joblib xgboost shap keras gensim


In [ ]:
# import nltk
# nltk.download('stopwords')

In [ ]:
# pip install gensim

In [ ]:
# dans cmd
# pip install tensorflow

In [ ]:
# Importer le fichier depuis le drive
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import re
import multiprocessing
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
import joblib
# df=df.rename(columns={"nom_client":"client","date_achat":"date_commande","date_avis":"date","text_avis":"Text","titre_avis":"Titre","note_avis":"star"})

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Drive non connecté")



Mounted at /content/drive


In [ ]:


try:
    model=joblib.load('/content/drive/MyDrive/Colab_supply_chain/Notebooks/w2v_4000_2.pkl')
except:
    model=joblib.load("w2v_4000_2.pkl")

try:
    df=pd.read_csv('/content/drive/MyDrive/Colab_supply_chain/Data/commentaires_prepares2.csv')
except:
    df=pd.read_csv("commentaires_prepares2.csv")


In [ ]:
model

In [ ]:
df.shape

(168897, 116)

In [ ]:

df=df.dropna(axis=0,how='any')
df=df.rename(columns={"Commentaire":"Text"})
df['Text'] = df['Text'].apply(lambda x: ' '.join(re.findall(r"'(\w+)'", x)))
display(df['Text'].head())
df.shape

0    jamais reçu commande écris showroomprive clôtu...
1    commandé collier lun ok lautre pa tout celui c...
2    commandé airpods reconditionnés dit excellent ...
3    produit acheté septembre retourné mois attends...
4    encore commande partiellement annulée dernier ...
Name: Text, dtype: object

(168892, 116)

In [ ]:
# Création d'une nouvelle variable "star_cluster" pour séparer le jeu de données en 2 segments,
# le segment des avis positifs (=1) et le segment des avis négatifs (=0)

# df['Sentiment'] = df['star']
df['Sentiment'] = df['star'].apply(lambda x: 1 if x >= 4 else 0)
df=df[["Text","Sentiment"]]
df["Text"].fillna('', inplace=True)
df.shape

<ipython-input-8-2bf3f6ab681e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Text"].fillna('', inplace=True)


(168892, 2)

In [ ]:
# Ca créer une liste qui contient des phrases tokenisées, représentées en liste de mots.
sentences = [text_to_word_sequence(comment) for comment in df['Text']]


X = []
# Suite à des problèmes de mots qui ne sont pas retrouvés dans le modèle, ça recupère, le poids des mots de chaque phrase,
# et stocke la moyenne des vecteurs, sauf si le mots n'est pas retrouvé dans le modèle, ce qui stocke le vecteur 0 pour la phrase
for words in sentences:
    vectors = [model[word] for word in words if word in model]
    if vectors:
      X.append(np.mean(vectors, axis=0))
    else:
      X.append(np.zeros(model.vector_size))  # Utiliser un vecteur nul si aucun mot valide n'est présent

# Cela transforme la liste en une colonne, ça concatène verticalement
X = np.vstack(X)
y = df['Sentiment']

from sklearn.decomposition import PCA

# Appliquer l'ACP pour réduire la dimensionnalité
pca = PCA(n_components = 0.9)
X_pca = pca.fit_transform(X)
print("Nombre de composantes retenues :", pca.n_components_)



# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.25, random_state=42)

Nombre de composantes retenues : 138


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Calcul et affichage de classification_report
print( classification_report(y_test, y_pred) )

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

              precision    recall  f1-score   support

           0       0.84      0.77      0.80     16901
           1       0.86      0.90      0.88     25322

    accuracy                           0.85     42223
   macro avg       0.85      0.84      0.84     42223
weighted avg       0.85      0.85      0.85     42223



Classe prédite,0,1
Classe réelle,,
0,13096,3805
1,2574,22748


In [ ]:
model.most_similar(negative=["livraison"])

[('sachet', 0.23088617622852325),
 ('casser', 0.22622404992580414),
 ('rapidité', 0.21335779130458832),
 ('complètement', 0.21328598260879517),
 ('gardé', 0.21136388182640076),
 ('entourage', 0.20842692255973816),
 ('verre', 0.20212896168231964),
 ('souple', 0.2011796236038208),
 ('rapide', 0.19644862413406372),
 ('plait', 0.19479654729366302)]

In [ ]:
model.wv.similarity("satisfait", 'content')

In [ ]:
model.wv.most_similar(positive=["satisfait", "content"], negative=["livraison"], topn=3)

In [ ]:
import xgboost as xgb

# X_train = np.asarray(X_train)
# y_train = np.asarray(y_train)
# X_test = np.asarray(X_test)

# param_grid = {
#     'n_estimators': [1000,4000],
#     'learning_rate': [0.1],
#     'max_depth': [9]
# }

clf = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    n_jobs=-1,
    )

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#  Importer la classe classification_report
from sklearn.metrics import classification_report

# Calcul et affichage de classification_report
print( classification_report(y_test, y_pred) )

display(clf.score(X_train, y_train))
display(clf.score(X_test, y_test))

# Calcul et affichage de la matrice de confusion
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix


              precision    recall  f1-score   support

           0       0.84      0.83      0.84     16901
           1       0.89      0.89      0.89     25322

    accuracy                           0.87     42223
   macro avg       0.86      0.86      0.86     42223
weighted avg       0.87      0.87      0.87     42223



0.8938887967853224

0.8696445065485636

Classe prédite,0,1
Classe réelle,,
0,14073,2828
1,2676,22646


In [ ]:
display(clf.score(X_train, y_train))
display(clf.score(X_test, y_test))

0.8828995255350559

0.8651445894417734

In [ ]:
# Récupération des scores d'accuracy
accuracies = grid_search.cv_results_['mean_test_score']

# Création du graphique
plt.plot(range(len(accuracies)), accuracies)
plt.xlabel('Test')
plt.ylabel('Accuracy')
plt.title('Scores d\'accuracy de chaque test de la validation croisée')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = clf.predict(X_test)
data = pd.DataFrame({'star': y_test.values})
data['y_pred'] = y_pred
fig, ax = plt.subplots(figsize=[15,8])
sns.violinplot(x="star", y="y_pred", data=data, ax=ax)
plt.show()

In [ ]:
import joblib

# joblib.dump(rf, 'w2v_rf.pkl')
joblib.dump(pca, 'w2v_pca_4000_v2.pkl')
joblib.dump(clf, 'w2v_clf_pca_4000_200_01_5_v2.pkl')

['w2v_clf_pca_4000_200_01_5_v2.pkl']

In [ ]:
import joblib

joblib.dump(rf, 'w2v_rf.pkl')
joblib.dump(pca, 'w2v_pca.pkl')
joblib.dump(clf, 'w2v_clf.pkl')

# Charger le modèle sauvegardé
loaded_model = joblib.load('w2v_clf.pkl')

# Commentaires pour lesquels vous souhaitez effectuer des prédictions
comments = ["très déçus", "Je ne recommande pas"]

# Appliquer la même prétraitement que vous avez effectué sur les données d'entraînement (par exemple, le nettoyage des commentaires et la tokenisation)
preprocessed_comments = [text_to_word_sequence(comment) for comment in comments]

# Calculer les vecteurs d'embedding pour chaque commentaire
comment_vectors = []
for words in preprocessed_comments:
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        comment_vectors.append(np.mean(vectors, axis=0))
    else:
        comment_vectors.append(np.zeros(model.vector_size))

# Empiler les vecteurs pour obtenir un tableau 2D
comment_vectors = np.vstack(comment_vectors)

# Faire les prédictions sur les commentaires
predicted_sentiments = loaded_model.predict(comment_vectors)

# Afficher les prédictions
for comment, sentiment in zip(comments, predicted_sentiments):
    print(f"Commentaire: {comment}")
    print(f"Sentiment prédit: {'Positive' if sentiment == 1 else 'Negative'}")
    print()

In [ ]:
importance = clf.feature_importances_
features_names = list(range(len(importance)))  # Remplacez cette liste par les noms de vos fonctionnalités si disponibles

plt.figure(figsize=(12, 6))
plt.bar(features_names, importance)
plt.xlabel('Fonctionnalité')
plt.ylabel('Importance')
plt.title('Importance des fonctionnalités dans le modèle XGBoost')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import shap

explainer = shap.Explainer(clf)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
model.most_similar(negative=["livraison","rapide"])